In [25]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/training_file.json") as f:
    data = pd.DataFrame(js.loads(line) for line in f)
f.close()

with open("/u/pmpande/author_profiling/testing_file.json") as ft:
    data_test = pd.DataFrame(js.loads(line) for line in ft)
ft.close()

In [29]:
import random
from math import sqrt
import numpy as np

import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from nltk.stem import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import mean_squared_error

acc1 = []
acc2 = []
acc3 = []
acc4 = []
rmse1 = []
rmse2 = []
rmse3 = []
rmse4 = []

for i in range(10):
    print("Running Iteration " + str(i) + "...")
    #Get Random 50,000 records for training
    rand_train = random.sample(range(0,len(data)),10000)
    train = data.iloc[rand_train].copy()

    #Get Random 5,000 records for testing
    rand_test = random.sample(range(0,len(data_test)),1000)
    test = data_test.iloc[rand_test].copy()
    
    corpus = []
    truth = []
    for row in train.iterrows():
        corpus.append(row[1]['text'])
        truth.append(row[1]['stars'])     

    corpus_test = []
    truth_test = []
    for row in test.iterrows():
        corpus_test.append(row[1]['text'])
        truth_test.append(row[1]['stars'])
        
    new = []
    count = 0
    for row in corpus:
        lst = []
        tokens = nltk.word_tokenize(row)
        tags = nltk.pos_tag(tokens)
        for tag in tags:
            if tag[1] == 'JJ':
                temp = (tag[0],'a')
                lst.append(temp)
            elif tag[1] == 'RB': 
                temp = (tag[0],'r')
                lst.append(temp)
        pos = 0
        neg = 0
        score = []
        for word in lst:
            if len(list(swn.senti_synsets(word[0],word[1]))) != 0:
                pos += (list(swn.senti_synsets(word[0],word[1]))[0]).pos_score()
                neg += (list(swn.senti_synsets(word[0],word[1]))[0]).neg_score()
        total = pos+neg
        if total == 0:
            score.append(0)
            score.append(0)
        else:
            score.append(pos/total)
            score.append(neg/total)
        new.append(score)
        
    new_test = []
    for row in corpus_test:
        lst = []
        tokens = nltk.word_tokenize(row)
        tags = nltk.pos_tag(tokens)
        for tag in tags:
            if tag[1] == 'JJ':
                temp = (tag[0],'a')
                lst.append(temp)
            elif tag[1] == 'RB': 
                temp = (tag[0],'r')
                lst.append(temp)
        pos = 0
        neg = 0
        score = []
        for word in lst:
            if len(list(swn.senti_synsets(word[0],word[1]))) != 0:
                pos += (list(swn.senti_synsets(word[0],word[1]))[0]).pos_score()
                neg += (list(swn.senti_synsets(word[0],word[1]))[0]).neg_score()
        total = pos+neg
        if total == 0:
            score.append(0)
            score.append(0)
        else:
            score.append(pos/total)
            score.append(neg/total)
        new_test.append(score)
        
    train_x = np.array(new)
    test_x = np.array(new_test)
        
    clf = MultinomialNB().fit(train_x,truth)
    clf1 = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42).fit(train_x,truth)
    clf2 = tree.DecisionTreeClassifier().fit(train_x,truth)
    clf3 = LogisticRegression(solver='newton-cg', multi_class='multinomial').fit(train_x,truth)

    predicted = clf.predict(test_x)
    acc1.append(np.mean(predicted == truth_test)) 

    predicted1 = clf1.predict(test_x)
    acc2.append(np.mean(predicted1 == truth_test))

    predicted2 = clf2.predict(test_x)
    acc3.append(np.mean(predicted2 == truth_test))

    predicted3 = clf3.predict(test_x)
    acc4.append(np.mean(predicted3 == truth_test)) 

    #targetNames = ['1','2','3','4','5']
    #print(metrics.classification_report(truth_test, predicted,target_names=targetNames))
    #print(metrics.classification_report(truth_test, predicted1,target_names=targetNames))
    #print(metrics.classification_report(truth_test, predicted2,target_names=targetNames))
    #print(metrics.classification_report(truth_test, predicted3,target_names=targetNames))
    
    #print(metrics.confusion_matrix(truth_test, predicted))
    #print("---------------------------------")
    #print(metrics.confusion_matrix(truth_test, predicted1))
    #print("---------------------------------")
    #print(metrics.confusion_matrix(truth_test, predicted2))
    #print("---------------------------------")
    #print(metrics.confusion_matrix(truth_test, predicted3))

    rmse1.append(sqrt(mean_squared_error(truth_test, predicted)))
    rmse2.append(sqrt(mean_squared_error(truth_test, predicted1)))
    rmse3.append(sqrt(mean_squared_error(truth_test, predicted2)))
    rmse4.append(sqrt(mean_squared_error(truth_test, predicted3)))
    
    del train
    del test


Running Iteration 0...
Running Iteration 1...
Running Iteration 2...
Running Iteration 3...
Running Iteration 4...
Running Iteration 5...
Running Iteration 6...
Running Iteration 7...
Running Iteration 8...
Running Iteration 9...


In [30]:
accf1 = 0
accf2 = 0
accf3 = 0
accf4 = 0
rmsef1 = 0
rmsef2 = 0
rmsef3 = 0
rmsef4 = 0
for i in range(10):
    accf1 += acc1[i]
    accf2 += acc2[i]
    accf3 += acc3[i]
    accf4 += acc4[i]
    
    rmsef1 += rmse1[i]
    rmsef2 += rmse2[i]
    rmsef3 += rmse3[i]
    rmsef4 += rmse4[i]
    
print("Average accuracy 1: " + str(accf1/10))
print("Average accuracy 2: " + str(accf2/10))
print("Average accuracy 3: " + str(accf3/10))
print("Average accuracy 4: " + str(accf4/10))
   
print("Average rmse 1: " + str(rmsef1/10))
print("Average rmse 2: " + str(rmsef2/10))
print("Average rmse 3: " + str(rmsef3/10))
print("Average rmse 4: " + str(rmsef4/10))

Average accuracy 1: 0.4232
Average accuracy 2: 0.4222
Average accuracy 3: 0.4102
Average accuracy 4: 0.4501
Average rmse 1: 1.8785850068325654
Average rmse 2: 1.6922369058116793
Average rmse 3: 1.7053672017600323
Average rmse 4: 1.7608033637464122
